In [1]:
pip install yfinance pandas_ta backtesting pandas numpy > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [1]:
import datetime
import pandas_ta as ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf
import numpy as np
import multiprocessing

multiprocessing.set_start_method('fork')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df = yf.download(tickers='AAPL', start='2019-01-01', end='2022-12-31')

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [' '.join(col).strip() for col in df.columns]
df.columns = [col.split(' ')[0] for col in df.columns]

df['RSI'] = ta.rsi(close=df['Close'], length=14)
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)

bbands = ta.bbands(close=df['Close'], length=30, std=2)
if isinstance(bbands, pd.DataFrame):
    df['BBL'] = bbands['BBL_30_2.0']
    df['BBM'] = bbands['BBM_30_2.0']
    df['BBU'] = bbands['BBU_30_2.0']
    df['BBB'] = (df['BBU'] - df['BBL']) / df['BBM']

df = df.dropna()
df.head()

[*********************100%***********************]  1 of 1 completed


,Adj,Close,High,Low,Open,Volume,RSI,ATR,BBL,BBM,BBU,BBB
Date,,,,,,,,,,,,
2019-02-13 00:00:00+00:00,40.810356,42.544998,43.119999,42.480000,42.847500,89960800,59.150999,0.975334,35.344983,39.796083,44.247184,0.223695
2019-02-14 00:00:00+00:00,40.959034,42.700001,42.814999,42.345001,42.427502,87342800,59.938535,0.939239,35.334253,39.903417,44.472580,0.229011
2019-02-15 00:00:00+00:00,40.867908,42.605000,42.924999,42.437500,42.812500,98507200,59.185391,0.906972,35.768408,40.138667,44.508925,0.217758
2019-02-19 00:00:00+00:00,40.990208,42.732498,42.860001,42.372501,42.427502,75891200,59.913393,0.877009,36.015530,40.327583,44.639637,0.213851
2019-02-20 00:00:00+00:00,41.253994,43.007500,43.330002,42.747501,42.797501,104457600,61.508168,0.857045,36.297800,40.528417,44.759033,0.208773


## Step 3: Trading Strategy Definition
- Define buy and sell conditions in a numpy series
- Create strat and use ATR to set stop loss and take profit


In [3]:
def apply_total_signal(df, rsi_threshold_low=30, rsi_threshold_high=70, bb_width_threshold=0.0015):
    signals = np.zeros(len(df))
    prev_close = df['Close'].shift(1)
    prev_high = df['High'].shift(1)
    prev_low = df['Low'].shift(1)

    buy_conditions = (
        (prev_close < df['BBL'].shift(1)) & 
        (df['RSI'].shift(1) < rsi_threshold_low) &  
        (df['Close'] > prev_high) &  
        (df['BBB'] > bb_width_threshold)  
    )

    sell_conditions = (
        (prev_close > df['BBU'].shift(1)) &  
        (df['RSI'].shift(1) > rsi_threshold_high) &  
        (df['Close'] < prev_low) &  
        (df['BBB'] > bb_width_threshold)  
    )

    signals[buy_conditions] = 2
    signals[sell_conditions] = 1

    return pd.Series(signals, index=df.index)


In [4]:
class MyStrat(Strategy):
    mysize = 0.1
    slcoef = 3.0
    TPcoef = 2.0
    rsi_threshold_low = 30
    rsi_threshold_high = 70
    bb_width_threshold = 0.0015

    def init(self):
        super().init()
        self.signal1 = self.I(self.calculate_signal)
        self.I(lambda: self.data.df['RSI'], name='RSI')

    def calculate_signal(self):
        return apply_total_signal(
            df=self.data.df,
            rsi_threshold_low=self.rsi_threshold_low,
            rsi_threshold_high=self.rsi_threshold_high,
            bb_width_threshold=self.bb_width_threshold
        )

    def next(self):
        super().next()
        if not self.position:
            slatr = self.slcoef * self.data.ATR[-1]
            tpatr = self.TPcoef * self.data.ATR[-1]

            if self.signal1[-1] == 2:  
                self.buy(sl=self.data.Close[-1] - slatr, tp=self.data.Close[-1] + tpatr, size=self.mysize)
            elif self.signal1[-1] == 1:  
                self.sell(sl=self.data.Close[-1] + slatr, tp=self.data.Close[-1] - tpatr, size=self.mysize)


## Step 4: Backtest and Optimization
Execute the backtest using the `Backtesting` library, and optimize for the best parameters.


In [6]:
bt = Backtest(df, MyStrat, cash=10000, margin=1/10, commission=0.001)
stats = bt.optimize(
    rsi_threshold_low=[25, 27, 30, 32, 35],
    rsi_threshold_high=[65, 67, 70, 72, 75],
    bb_width_threshold=[0.001, 0.002, 0.005, 0.01, 0.015],
    slcoef=[2.0, 2.5, 3.0, 3.5, 4.0],
    TPcoef=[2.5, 3.0, 3.5, 4.0, 4.5],
    maximize='Return [%]'
)
print(stats)
bt.plot()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 3125 configurations.
  output = _optimize_grid()


Start                     2019-02-13 00:00...
End                       2022-12-30 00:00...
Duration                   1416 days 00:00:00
Exposure Time [%]                   30.745659
Equity Final [$]                 14480.333438
Equity Peak [$]                  15160.660912
Return [%]                          44.803334
Buy & Hold Return [%]              205.394284
Return (Ann.) [%]                    9.998127
Volatility (Ann.) [%]               20.155809
Sharpe Ratio                         0.496042
Sortino Ratio                        0.796676
Calmar Ratio                         0.333307
Max. Drawdown [%]                  -29.996793
Avg. Drawdown [%]                   -3.936532
Max. Drawdown Duration      616 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   11
Win Rate [%]                        45.454545
Best Trade [%]                      30.314092
Worst Trade [%]                    -15.801921
Avg. Trade [%]                    

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='1543', ...)